# TP2: Redes Recurrentes y Representaciones Incrustadas

# 1. (100 puntos) Red LSTM

## Imports

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
import numpy as np

## Constantes

In [24]:
TEST_SIZE = 0.4 # Define % del dataset que será utilizado para el dataset de pruebas que se retornará en el método implementado load_process_data
MAX_WORDS = 200 # Establece el TOP de palabras que se utilizan en el diccionario de palabras, el tamaño de la representación incrustada.
BATCH_SIZE = 64 # Tamaño del batch que se utilizará para el dataset de entrenamiento.
COLLECTION_PATH = ".\\smsspamcollection" # Ruta base para encontrar el dataset de pruebas de spam.

## Funciones requeridas

### Código Provisto

Se corresponde con el código encontrado en el archivo "Natural_disaster_NLP_LSTM.ipynb", del cual se realizaron las modificaciones y consideraciones que fueron mencionadas durante las clases del curso.

#### Data mapper

In [4]:
class DatasetMaper(Dataset):
    '''
    Handles batches of dataset
    '''

    def __init__(self, x, y):
        """
        Inits the dataset mapper
        """
        self.x = x
        self.y = y

    def __len__(self):
        """
        Returns the length of the dataset
        """
        return len(self.x)

    def __getitem__(self, idx):
        """
        Fetches a specific item by id
        """
        return self.x[idx], self.y[idx]

---
#### Modelo de entranamiento

In [5]:
class LSTM_TweetClassifier(nn.ModuleList):

    def __init__(self, batch_size=BATCH_SIZE, hidden_dim=20, lstm_layers=2, max_words=MAX_WORDS):
        """
        param batch_size: batch size for training data
        param hidden_dim: number of hidden units used in the LSTM and the Embedding layer
        param lstm_layers: number of lstm_layers
        param max_words: maximum sentence length
        """
        super(LSTM_TweetClassifier, self).__init__()
        # batch size during training
        self.batch_size = batch_size
        # number of hidden units in the LSTM layer
        self.hidden_dim = hidden_dim
        # Number of LSTM layers
        self.LSTM_layers = lstm_layers
        self.input_size = max_words  # embedding dimension

        self.dropout = nn.Dropout(0.5)  # Para descartar
        #  N, D			#  hidden_dim -> Determina el tamaño del embedding
        self.embedding = nn.Embedding(self.input_size, self.hidden_dim, padding_idx=0)  # Aprender la representacion
        self.lstm = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers,
                            batch_first=True)  # Capaz de aprender/olvidar dependiendo de las relaciones.
        self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=257)
        self.fc2 = nn.Linear(257, 1)

    def forward(self, x):
        """
        Forward pass
        param x: model input
        """
        # it starts with noisy estimations of h and c
        #  Context y estado
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))  # "Contexto"
        c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))  # "Estado"
        # Fills the input Tensor with values according to the method described in Understanding the difficulty of training deep feedforward neural networks - Glorot, X. & Bengio, Y. (2010), using a normal distribution.
        # The resulting tensor will have values sampled from \mathcal{N}(0, \text{std}^2)N(0,std)
        torch.nn.init.xavier_normal_(h)
        torch.nn.init.xavier_normal_(c)
        # print("x shape ", x.shape)
        # print("embedding ", self.embedding)
        out = self.embedding(x)
        out, (hidden, cell) = self.lstm(out, (h, c))
        out = self.dropout(out)

        #  Fully connected network para la clasificacion
        out = torch.relu_(self.fc1(out[:, -1, :]))
        out = self.dropout(out)
        # sigmoid activation function
        out = torch.sigmoid(self.fc2(out))

        return out

### Código base implementado

---
#### Preprocesamientos

Métodos de preprocesado implementados según indicaciones del punto 1.a del TP2. Esto incluye la implementación de los métodos *preprocesar_documento_1*, *preprocesar_documento_2* y *preprocess_example*. Este último tiene por objetivo presentar la ejecución de los dos primeros métodos citados en la sección de *Resolución de ejercicios* de más adelante.

In [59]:
def preprocesar_documento_1(document, to_string=False):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(document)
    word_tokens = [token.lower() for token in word_tokens]
    word_tokens = [token for token in word_tokens if token.isalnum()]  # To remove punctuations
    filtration = [word for word in word_tokens if word not in stop_words]
    return filtration if not to_string else ' '.join(filtration)

def preprocesar_documento_2(document, to_string=False):
    word_tokens = preprocesar_documento_1(document)
    lemmatizer = WordNetLemmatizer()
    filtration = [lemmatizer.lemmatize(word, pos="v") for word in word_tokens]
    return filtration if not to_string else ' '.join(filtration)

def preprocess_example():
    with open(COLLECTION_PATH + "\\SMSSpamCollection", 'r') as collection:
        for line in collection:
            d0 = line.replace("ham", "").replace("spam", "").replace("\n", "").replace("\t", "")
            d1 = "I thought, I thought of thinking of thanking you for the gift"
            d2 = "She was thinking of going to go and get you a GIFT!"
            
            print("Testing preprocessing methods with different lines:\n\n{}\n{}\n{}".format(print_test(1, d0), print_test(2, d1), print_test(3, d2)))
            break

def print_test(test, line):
    return "* Test line #{}: {}\nPreprocess #1: {}\nPreprocess #2: {}\n".format(test, line, preprocesar_documento_1(line, to_string=True), preprocesar_documento_2(line, to_string=True))

---
#### Tokenizer

Los métodos *tokens_to_indexes*, *sequence_to_number_combination* y *adapt_to_input_layer* son implementaciones generadas como "equivalentes" para las funciones implementadas en el código provisto llamadas *prepare_tokens* y *sequence_to_token*. Su objetivo principal es, en tres pasos bien definidos por método:

1. establecer el *diccionario* del TOP *MAX_WORDS* (cuya explicación se encuentra en la sección de *Constantes*) para la posterior definición de representación numérica para las líneas preprocesadas.
2. generar la representación equivalente de cada línea según los valores numéricos obtenidos del diccionario obtenido en el punto anterior (se ignoran aquellas palabras que no estén en el diccionario).
3. adaptar la representación obtenida del punto anterior al tamaño *MAX_WORDS* para que coincida con la entrada para el entrenamiento de este punto.

In [8]:
def tokens_to_indexes(sentences: list) -> dict:
    words_count = {}
    for sentence in sentences:
        for word in sentence:
            if word in words_count:
                words_count[word] += 1
            else:
                words_count[word] = 1
    words_to_list = list(dict(sorted(words_count.items(), key=lambda item: item[1])))
    words_to_list.reverse()
    top_max_words = words_to_list[0: MAX_WORDS - 1]
    index_words = {}
    counter = 1
    for word in top_max_words:
        index_words[word] = counter
        counter += 1
    return index_words

def sequence_to_number_combination(word: list, index_words: dict):
    sequence = []
    for token in word:
        if token in index_words:
            sequence.append(index_words[token])
    return sequence

def adapt_to_input_layer(dataset, input_layer_size=MAX_WORDS):
    new_dataset = []
    for data in dataset:
        zeros_to_add = input_layer_size - len(data)
        new_data_list = [0 for zero in range(zeros_to_add)]
        new_data_list.extend(data)
        new_dataset.append(new_data_list)
    return new_dataset

---
#### Data Loader

Este método concentra todo lo discutido anteriormente y termina por generar los datasets de entrenamiento y pruebas por utilizar en la siguiente sección de *Resolución de ejercicios*, más específicamente en los puntos 1.b y 1.c.

In [25]:
def load_process_data(f_prepros):
    # Load dataset
    dataset_frame = pd.read_csv('smsspamcollection\\SMSSpamCollection', delimiter='\t', header=None)
    # Preprocess document
    sentences_list = [f_prepros(sentence) for sentence in dataset_frame[1]]
    # Add index to every word
    words_dictionary = tokens_to_indexes(sentences_list)
    # Transform tokens (words) to indexes (numbers)
    sentences_list = [sequence_to_number_combination(sentence, words_dictionary) for sentence in sentences_list]
    # One-hot tags ham = 0, spam = 1
    tags_list = [0 if tag == "ham" else 1 for tag in dataset_frame[0]]
    # Build train and test datasets
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(sentences_list, tags_list, test_size=TEST_SIZE)
    # Adapt data to input layer
    x_train = adapt_to_input_layer(X_train_raw)
    x_test = adapt_to_input_layer(X_test_raw)
    training_set = DatasetMaper(x_train, y_train)
    test_set = DatasetMaper(x_test, y_test)
    loader_training = DataLoader(training_set, batch_size=BATCH_SIZE)
    loader_test = DataLoader(test_set)
    return loader_training, loader_test

## Resolución de ejercicios

En esta sección, haciendo uso del código provisto e implementado anteriormente, se dará resolución a los ejercicios propuestos en el punto #1 del TP2.

### 1. Implemente la siguiente arquitectura de una red LSTM:

#### a.1 Muestre un ejemplo con una entrada escogida del pre-procesamiento con ambos enfoques, y explique brevemente los posibles efectos de utilizar el segundo enfoque al primero. 

In [63]:
preprocess_example()

Testing preprocessing methods with different lines:

* Test line #1: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Preprocess #1: go jurong point crazy available bugis n great world la e buffet cine got amore wat
Preprocess #2: go jurong point crazy available bugis n great world la e buffet cine get amore wat

* Test line #2: I thought, I thought of thinking of thanking you for the gift
Preprocess #1: thought thought thinking thanking gift
Preprocess #2: think think think thank gift

* Test line #3: She was thinking of going to go and get you a GIFT!
Preprocess #1: thinking going go get gift
Preprocess #2: think go go get gift



**R/** En el primer preprocesamiento se emplea únicamente la eliminación de mayúsculas, signos de puntuación y *stopwords*. Estas últimas, son palabras cuya significancia es mínima en el procesamiento de lenguaje natural por lo que es posible retirarlas sin generar mayor impacto en el entendimiento de las expresiones. Como se puede observar en las anteriores entradas, su funcionamiento consiste en estandarizar la expresión al retirar mayúsculas y la puntuación, además de eliminar preposiciones y palabras que sean entendidas como *stopwords* por *nltk*. Puede observarse en el *test line #1* como la palabra *until* desaparece al ser preprocesada. Entre los ejemplos que se muestran, el *test line #1* elimina palabras como *until*, *only* e *in*, mientras que en el *test line #2* también se retiran *of*, *you*, *for* y *the*. Esto evidentemente reduce el espectro de palabras a tomar en cuenta para el procesamiento.

En el segundo preprocesamiento, aparte de aplicar primero lo expuesto anteriormente, se emplea también la *lematización* de la expresión. Esto permite obtener la *raíz* de una palabra a partir de cualquiera de sus variables posibles, tal como se puede observar en los ejemplos anteriores donde *thinking* pasa a su forma base *think*. Además de eso, otras funciones con las que cuenta es reducir las palabras a su forma singular, cambiar el tiempo, entre otros. Por ejemplo, en el *test line #1* se cambia el verbo en pasado *got* por el verbo *get*, en el *test line #2* se cambian las palabras *thought*, *thinking* y *thanking* por *think*, *think* y *thank* y, finalmente, en el *test line #3* se cambian las palabras *thinking* y *going* por *think* y *go*.

Los efectos individuales que se identifican para el primer preprocesamiento consiste en estandarizar todas las frases del dataset por utilizar. Esto igualmente mantiene palabras que son iguales pero que pueden estar conjugadas, haciendo que al ser procesadas se interpreten como palabras distintas, afectando el entrenamiento y posterior prueba utilizando frases muy similares pero que estén conjugadas en otros tiempos, solo por dar un ejemplo.

Esto precisamente es lo que se busca con la especialización del método #1, es decir, el segundo preprocesamiento. Este no solo aprovecha las virtudes de estandarización del primer preprocesamiento, sino que también incluye la lematización, lo cual reduce aún más la gama de palabras a utilizar para el análisis, produciendo que tanto al entrenar como probar se encuentre que las palabras *get* y *got*, aunque conjugadas en distinto tiempo, corresponden a la misma palabra y tienen un significado y aplicación relativamente similares. El efecto que se identifica es que optimizará la ejecución del entrenamiento del modelo y favorecerá una mayor efectividad al momento de predecir ya que no tomará las conjugaciones y otras variantes de las palabras como palabras completamente distintas, haciendo que ahora se les pueda dar más sentido al interpretar que son la misma palabra.

In [ ]:
load_process_data(preprocesar_documento_1)
load_process_data(preprocesar_documento_2)